Simulate Minkowski with backreaction, as a consistency check. Perform simulations with different Pauli-Villars mass, and for each choice of mass, perform simulations with increasing kmax, and lmax. The expectation is that with more quantum modes evolved, the solution will be closer to Minkowski. To quantify that we construct L2 norms of the solutions (the classical state vector), minus the Minkowski solution.

You need to run with backreaction and save "data".

In [ ]:
using HDF5
using LaTeXStrings
using Plots ; pythonplot()
using DelimitedFiles
using SpheriCo

In [ ]:
# give the directory where the data from all the runs are saved
dir = "../examples/quantum_runs/"

par1 = "a0.0_b0.0_c1.0_rmax30.0_tmax10.0_cfl0.0625_sigma0.0_overMp2_0.9999889550967277_reg_true_backreact_true_rcut20.0_mPV0.1_dk_denom_30_kmax5.0_lmax15.0"
par2 = "a0.0_b0.0_c1.0_rmax30.0_tmax10.0_cfl0.0625_sigma0.0_overMp2_0.9999889550967277_reg_true_backreact_true_rcut20.0_mPV0.1_dk_denom_30_kmax10.0_lmax30.0"
par3 = "a0.0_b0.0_c1.0_rmax30.0_tmax10.0_cfl0.0625_sigma0.0_overMp2_0.9999889550967277_reg_true_backreact_true_rcut20.0_mPV0.1_dk_denom_30_kmax20.0_lmax60.0"

par4 = "a0.0_b0.0_c1.0_rmax30.0_tmax10.0_cfl0.0625_sigma0.0_overMp2_0.9988967160535545_reg_true_backreact_true_rcut20.0_mPV1.0_dk_denom_30_kmax5.0_lmax15.0"
par5 = "a0.0_b0.0_c1.0_rmax30.0_tmax10.0_cfl0.0625_sigma0.0_overMp2_0.9988967160535545_reg_true_backreact_true_rcut20.0_mPV1.0_dk_denom_30_kmax10.0_lmax30.0"
par6 = "a0.0_b0.0_c1.0_rmax30.0_tmax10.0_cfl0.0625_sigma0.0_overMp2_0.9988967160535545_reg_true_backreact_true_rcut20.0_mPV1.0_dk_denom_30_kmax20.0_lmax60.0"

par7 = "a0.0_b0.0_c1.0_rmax30.0_tmax10.0_cfl0.0625_sigma0.0_overMp2_0.9956014228528787_reg_true_backreact_true_rcut20.0_mPV2.0_dk_denom_30_kmax5.0_lmax15.0"
par8 = "a0.0_b0.0_c1.0_rmax30.0_tmax10.0_cfl0.0625_sigma0.0_overMp2_0.9956014228528787_reg_true_backreact_true_rcut20.0_mPV2.0_dk_denom_30_kmax10.0_lmax30.0"
par9 = "a0.0_b0.0_c1.0_rmax30.0_tmax10.0_cfl0.0625_sigma0.0_overMp2_0.9956014228528787_reg_true_backreact_true_rcut20.0_mPV2.0_dk_denom_30_kmax20.0_lmax60.0"

dir1 = dir*par1
dir2 = dir*par2
dir3 = dir*par3

dir4 = dir*par4
dir5 = dir*par5
dir6 = dir*par6

dir7 = dir*par7
dir8 = dir*par8
dir9 = dir*par9

# same for all dirs
D = 3
Nr = 128*2^D + 3 # the overal course graining
# load the r grid
r = h5read(dir1*"/data_$(Nr)/r.h5","r")
dr = r[2] - r[1]

out_dir = "./convergence_plots/minkowski_backreaction/"
if ispath(out_dir)==false
    mkpath(out_dir)
end

List the timesteps in all three batches.

In [ ]:
# list all available iterations (and corresponding files)
(its1, all_filenames1) = list_h5_files(dir1*"/data_$(Nr)", prefix="data_");
(its2, all_filenames2) = list_h5_files(dir2*"/data_$(Nr)", prefix="data_");
(its3, all_filenames3) = list_h5_files(dir3*"/data_$(Nr)", prefix="data_");

(its4, all_filenames4) = list_h5_files(dir4*"/data_$(Nr)", prefix="data_");
(its5, all_filenames5) = list_h5_files(dir5*"/data_$(Nr)", prefix="data_");
(its6, all_filenames6) = list_h5_files(dir6*"/data_$(Nr)", prefix="data_");

(its7, all_filenames7) = list_h5_files(dir7*"/data_$(Nr)", prefix="data_");
(its8, all_filenames8) = list_h5_files(dir8*"/data_$(Nr)", prefix="data_");
(its9, all_filenames9) = list_h5_files(dir9*"/data_$(Nr)", prefix="data_");

println("length(its3)=",length(its3))
#println("its1",its1)

println("length(its6)=",length(its6))
#println("its4",its4)

println("length(its9)=",length(its9))
#println("its7",its7)


Function to construct the L2 norm of the error.

In [ ]:
#v=["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]
# norm2 has size (number_of_different_resols, number_of_saved_timesteps)
function norm2(norm2, ri_max, ddir1, ddir2, ddir3)

    (its1, all_filenames1) = list_h5_files(ddir1*"/data_$(Nr)", prefix="data_");
    (its2, all_filenames2) = list_h5_files(ddir2*"/data_$(Nr)", prefix="data_");
    (its3, all_filenames3) = list_h5_files(ddir3*"/data_$(Nr)", prefix="data_");
    
    for i in 1:length(its3)

        it_data1 = its1[i]
        it_str_data1  = lpad(it_data1, 4, "0")
        v1 = h5read(ddir1*"/data_$(Nr)/data_$(it_str_data1).h5","v")
        t1 =  h5readattr(ddir1*"/data_$(Nr)/data_$(it_str_data1).h5", "/")["time"]
        
        it_data2 = its2[i]
        it_str_data2  = lpad(it_data2, 4, "0")
        v2 = h5read(ddir2*"/data_$(Nr)/data_$(it_str_data2).h5","v")
        t2 =  h5readattr(ddir2*"/data_$(Nr)/data_$(it_str_data2).h5", "/")["time"]

        it_data3 = its3[i]
        it_str_data3  = lpad(it_data3, 4, "0")
        v3 = h5read(ddir3*"/data_$(Nr)/data_$(it_str_data3).h5","v")
        t3 =  h5readattr(ddir3*"/data_$(Nr)/data_$(it_str_data3).h5", "/")["time"]
        
        # checks if time instant is the same for all types of data
        @assert t1 ≈ t2 ≈ t3

        #A
        A1 = v1[3:ri_max, 4] .- ones(length(r[3:ri_max])) 
        A2 = v2[3:ri_max, 4] .- ones(length(r[3:ri_max]))
        A3 = v3[3:ri_max, 4] .- ones(length(r[3:ri_max]))
        #B
        B1 = v1[3:ri_max, 5] .- ones(length(r[3:ri_max])) 
        B2 = v2[3:ri_max, 5] .- ones(length(r[3:ri_max]))
        B3 = v3[3:ri_max, 5] .- ones(length(r[3:ri_max]))
        #DB
        DB1 = v1[3:ri_max, 6] 
        DB2 = v2[3:ri_max, 6]
        DB3 = v3[3:ri_max, 6]
        #Utld
        Utld1 = v1[3:ri_max, 7] 
        Utld2 = v2[3:ri_max, 7]
        Utld3 = v3[3:ri_max, 7]
        #K
        K1 = v1[3:ri_max, 8] 
        K2 = v2[3:ri_max, 8]
        K3 = v3[3:ri_max, 8]
        #DB
        KB1 = v1[3:ri_max, 9] 
        KB2 = v2[3:ri_max, 9]
        KB3 = v3[3:ri_max, 9]
        #λ
        λ1 = v1[3:ri_max, 10] 
        λ2 = v2[3:ri_max, 10]
        λ3 = v3[3:ri_max, 10]
        #α
        α1 = v1[3:ri_max, 11] .- ones(length(r[3:ri_max])) 
        α2 = v2[3:ri_max, 11] .- ones(length(r[3:ri_max]))
        α3 = v3[3:ri_max, 11] .- ones(length(r[3:ri_max]))
        #Dα
        Dα1 = v1[3:ri_max, 12] 
        Dα2 = v2[3:ri_max, 12]
        Dα3 = v3[3:ri_max, 12]

        # norm2 = Sum(A^2 + B^2 + DB^2 + Utld^2 + K^2 + KB^2 + λ^2 + α^2 + Dα^2)*dr
        # where each variable above is understood as its difference from its Minkowski value
        norm2[1, i] = norm2[1, i] + dr*sum(A1.^2 .+ B1.^2 .+ DB1.^2 .+ Utld1.^2 .+ K1.^2 .+ KB1.^2 .+ λ1.^2 .+ α1.^2 + Dα1.^2)
        norm2[2, i] = norm2[2, i] + dr*sum(A2.^2 .+ B2.^2 .+ DB2.^2 .+ Utld2.^2 .+ K2.^2 .+ KB2.^2 .+ λ2.^2 .+ α2.^2 + Dα2.^2)
        norm2[3, i] = norm2[3, i] + dr*sum(A3.^2 .+ B3.^2 .+ DB3.^2 .+ Utld3.^2 .+ K3.^2 .+ KB3.^2 .+ λ3.^2 .+ α3.^2 + Dα3.^2)

    end # i loop

    return norm2
end

List of timesteps where data are saved, for the longest simulation.

In [ ]:
t_list = zeros(length(its1))
for i in 1:length(its1)
    it_data1 = its1[i]
    it_str_data1  = lpad(it_data1, 4, "0")
    t1 =  h5readattr(dir1*"/data_$(Nr)/data_$(it_str_data1).h5", "/")["time"]
    t_list[i] = t1
end
println("t_list = ", t_list)

ri_max is the index of the radial grid, until which you want to perform the comparison. Take care of which part is simulated well with backreaction included.

In [ ]:
ri_max = 345
println("r[ri_max] = ", r[ri_max])

Calculate L2 norms of errors (deviation of numerical solution from Minkowski).

In [ ]:
# M_PV = 0.1
norm2_all_MPV01 = zeros(3, length(its3))
norm2_all_MPV01 = norm2(norm2_all_MPV01, ri_max, dir1, dir2, dir3)

# M_PV = 1
norm2_all_MPV1 = zeros(3, length(its6))
norm2_all_MPV1 = norm2(norm2_all_MPV1, ri_max, dir4, dir5, dir6)

# M_PV = 2
norm2_all_MPV2 = zeros(3, length(its9))
norm2_all_MPV2 = norm2(norm2_all_MPV2, ri_max, dir7, dir8, dir9)

Plot the deviations:

In [ ]:
# M_PV = 0.1
ti1 = 2
tf1 = length(its3)
plot(t_list[ti1:tf1], sqrt.(norm2_all_MPV01[1,ti1:tf1]), label = L"|| \mathbf{u}||_{5, 15, 0.1}",
    linewidth = 2.8)
plot!(t_list[ti1:tf1], sqrt.(norm2_all_MPV01[2,ti1:tf1]), label = L"|| \mathbf{u}||_{10, 30, 0.1}",
    linewidth = 2.6, style=:dash)
p1 = plot!(t_list[ti1:tf1], sqrt.(norm2_all_MPV01[3,ti1:tf1]), label = L"|| \mathbf{u}||_{20, 60, 0.1}", 
    linewidth = 2.4, frame = true, style=:dot,
    legend=:topleft,  ylabel="norms", xaxis=nothing, legendfontsize=10, xlim=(0,9.96))

# M_PV = 1
ti2 = 2
tf2 = length(its6)
plot(t_list[ti2:tf2], sqrt.(norm2_all_MPV1[1,ti2:tf2]), label = L"|| \mathbf{u}||_{5, 15, 1}",
    linewidth = 2.8)
plot!(t_list[ti2:tf2], sqrt.(norm2_all_MPV1[2,ti2:tf2]), label = L"|| \mathbf{u}||_{10, 30, 1}",
    linewidth = 2.6, style=:dash)
p2 = plot!(t_list[ti2:tf2], sqrt.(norm2_all_MPV1[3,ti2:tf2]), label = L"|| \mathbf{u}||_{20, 60, 1}", 
    linewidth = 2.4, frame = true, style=:dot,
    legend=:topleft,  ylabel="norms", legendfontsize=10, xlim=(0,9.96), xaxis=nothing,)

# M_PV = 2
ti2 = 2
tf2 = length(its9)
plot(t_list[ti2:tf2], sqrt.(norm2_all_MPV2[1,ti2:tf2]), label = L"|| \mathbf{u}||_{5, 15, 2}",
    linewidth = 2.8)
plot!(t_list[ti2:tf2], sqrt.(norm2_all_MPV2[2,ti2:tf2]), label = L"|| \mathbf{u}||_{10, 30, 2}",
    linewidth = 2.6, style=:dash)
p3 = plot!(t_list[ti2:tf2], sqrt.(norm2_all_MPV2[3,ti2:tf2]), label = L"|| \mathbf{u}||_{20, 60, 2}", 
    linewidth = 2.4, frame = true, style=:dot,
    legend=:topleft,  ylabel="norms", xlabel="time", legendfontsize=10, xlim=(0,9.96))

plt = plot(p1, p2, p3, layout = grid(3, 1 ), wsize = (800,500))

savefig(plt, out_dir*"/norms_all.pdf")